In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

In [ ]:


# Charger les données KDD Cup 99
df = pd.read_csv("artifacts/KDDCup99.csv", header=None)

# Sélection de colonnes pertinentes
df = df.iloc[:, [0, 1, 4, 5]]
df.columns = ["duration", "protocol_type", "src_bytes", "dst_bytes"]

# Vérifier et convertir les colonnes en numérique
df["duration"] = pd.to_numeric(df["duration"], errors="coerce")
df["src_bytes"] = pd.to_numeric(df["src_bytes"], errors="coerce")
df["dst_bytes"] = pd.to_numeric(df["dst_bytes"], errors="coerce")

# Remplacer les NaN par 0
df.fillna(0, inplace=True)

# Encodage de "protocol_type"
label_encoder = LabelEncoder()
df["protocol_type"] = label_encoder.fit_transform(df["protocol_type"])

# Normalisation des données
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Entraîner Isolation Forest avec scoring
model = IsolationForest(contamination="auto", random_state=42)
model.fit(df_scaled)

# Prédire les anomalies avec score d'anomalie
df["anomaly"] = model.predict(df_scaled)
df["score"] = model.decision_function(df_scaled)  # Score d'anomalie

# Mapper les résultats
df["anomaly"] = df["anomaly"].map({1: "Normal", -1: "Anomalie"})

# ✅ Équilibrer les classes en réduisant les non-anomalies
anomalies = df[df["anomaly"] == "Anomalie"]
normaux = df[df["anomaly"] == "Normal"].sample(len(anomalies) * 2, random_state=42)  # Ratio 2:1

df_balanced = pd.concat([anomalies, normaux])

# Sauvegarde du modèle
joblib.dump(model, "./artifacts/isolation_forest.pkl")

# Affichage des résultats
print("🎯 Modèle entraîné et sauvegardé avec succès !")
print(df_balanced["anomaly"].value_counts())

# Génération d'alerte si des anomalies sont détectées
if len(anomalies) > 0:
    print(f"⚠️ {len(anomalies)} anomalies détectées ! Score moyen : {anomalies['score'].mean():.4f}")


C:\Users\nguye\AppData\Local\Temp\ipykernel_22508\1300564688.py:8: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("artifacts/KDDCup99.csv", header=None)


🎯 Modèle entraîné et sauvegardé avec succès !
anomaly
Normal      91718
Anomalie    45859
Name: count, dtype: int64
⚠️ 45859 anomalies détectées ! Score moyen : -0.0815
